# ⚛️ Módulo 3: Mecánica Molecular y Campos de Fuerza
## Actividad 3.3: Optimización de Geometrías Moleculares

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_03_mecanica_molecular/03_optimizacion_geometrias.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender qué es una optimización de geometría y por qué es importante
- Conocer los fundamentos matemáticos de la minimización de energía
- Diferenciar entre métodos de optimización locales y globales
- Aplicar métodos determinísticos: Steepest Descent, Conjugate Gradient, Newton-Raphson, L-BFGS
- Comprender métodos estocásticos: Monte Carlo, Simulated Annealing
- Implementar algoritmos genéticos y evolución diferencial
- Evaluar criterios de convergencia y calidad de optimización
- Realizar optimizaciones prácticas con RDKit y otros software
- Identificar y resolver problemas comunes en optimización

---

## 📚 Fundamentos Teóricos

### ¿Qué es la Optimización de Geometría?

La **optimización de geometría** es el proceso de encontrar la configuración atómica (posiciones de los átomos) que corresponde a un **mínimo de energía** en la superficie de energía potencial (PES).

> "Una molécula en su estado de equilibrio adopta la geometría que minimiza su energía potencial" - Principio fundamental de la química computacional

### Representación Matemática

Para un sistema molecular, la energía $E$ depende de las coordenadas atómicas $\mathbf{R} = (x_1, y_1, z_1, ..., x_N, y_N, z_N)$:

$$E = E(\mathbf{R})$$

El objetivo es encontrar $\mathbf{R}^*$ tal que:

$$E(\mathbf{R}^*) = \min_{\mathbf{R}} E(\mathbf{R})$$

### Gradiente y Hessiano

**Gradiente** (primera derivada): Vector que apunta en la dirección de máximo incremento de energía

$$\mathbf{g} = \nabla E = \left(\frac{\partial E}{\partial x_1}, \frac{\partial E}{\partial y_1}, ..., \frac{\partial E}{\partial z_N}\right)$$

En el mínimo: $\mathbf{g} = \mathbf{0}$ (todas las fuerzas son cero)

**Hessiano** (segunda derivada): Matriz que describe la curvatura de la PES

$$\mathbf{H}_{ij} = \frac{\partial^2 E}{\partial R_i \partial R_j}$$

En un mínimo verdadero: todos los autovalores de $\mathbf{H}$ deben ser positivos

### Tipos de Puntos Estacionarios

| Tipo | Gradiente | Hessiano | Descripción |
|------|-----------|----------|-------------|
| **Mínimo global** | $\mathbf{g} = \mathbf{0}$ | Definido positivo (todos $\lambda > 0$) | Menor energía absoluta |
| **Mínimo local** | $\mathbf{g} = \mathbf{0}$ | Definido positivo | Mínimo en una región |
| **Estado de transición** | $\mathbf{g} = \mathbf{0}$ | 1 autovalor negativo | Punto de silla de orden 1 |
| **Punto de silla** | $\mathbf{g} = \mathbf{0}$ | $\geq 2$ autovalores negativos | Punto de silla de orden superior |

### Referencias Fundamentales

1. **Schlegel, H. B.** (2011). *Geometry Optimization*. WIREs Comput. Mol. Sci., 1, 790-809. DOI: 10.1002/wcms.34

2. **Nocedal, J. & Wright, S. J.** (2006). *Numerical Optimization*, 2nd ed. Springer. ISBN: 978-0387303031

3. **Jensen, F.** (2017). *Introduction to Computational Chemistry*, 3rd ed. Wiley. Cap. 10-11.

4. **Leach, A. R.** (2001). *Molecular Modelling: Principles and Applications*, 2nd ed. Pearson. Cap. 7.

## 🎯 Métodos de Optimización Local

Los métodos locales buscan el mínimo más cercano a la geometría inicial.

### 1. Steepest Descent (Descenso Más Pronunciado)

**Principio:** Moverse en la dirección opuesta al gradiente (dirección de máxima reducción de energía)

$$\mathbf{R}_{n+1} = \mathbf{R}_n - \alpha \mathbf{g}_n$$

Donde:
- $\alpha$ = tamaño de paso (learning rate)
- $\mathbf{g}_n$ = gradiente en el paso $n$

**Ventajas:**
- ✅ Muy simple de implementar
- ✅ Bajo costo computacional por paso
- ✅ Robusto para geometrías iniciales malas
- ✅ No requiere calcular el Hessiano

**Desventajas:**
- ❌ Convergencia muy lenta cerca del mínimo
- ❌ Puede oscilar en valles estrechos
- ❌ Requiere muchas iteraciones

**Cuándo usar:**
- Geometrías iniciales muy alejadas del mínimo
- Pre-optimización antes de métodos más sofisticados
- Sistemas muy grandes donde calcular el Hessiano es prohibitivo

### 2. Conjugate Gradient (Gradiente Conjugado)

**Principio:** Mejora sobre steepest descent usando direcciones de búsqueda conjugadas

$$\mathbf{R}_{n+1} = \mathbf{R}_n + \alpha_n \mathbf{d}_n$$

$$\mathbf{d}_n = -\mathbf{g}_n + \beta_n \mathbf{d}_{n-1}$$

**Fórmulas para $\beta$ (variantes):**

**Fletcher-Reeves:**
$$\beta_n^{FR} = \frac{\mathbf{g}_n^T \mathbf{g}_n}{\mathbf{g}_{n-1}^T \mathbf{g}_{n-1}}$$

**Polak-Ribière:**
$$\beta_n^{PR} = \frac{\mathbf{g}_n^T (\mathbf{g}_n - \mathbf{g}_{n-1})}{\mathbf{g}_{n-1}^T \mathbf{g}_{n-1}}$$

**Hestenes-Stiefel:**
$$\beta_n^{HS} = \frac{\mathbf{g}_n^T (\mathbf{g}_n - \mathbf{g}_{n-1})}{\mathbf{d}_{n-1}^T (\mathbf{g}_n - \mathbf{g}_{n-1})}$$

**Ventajas:**
- ✅ Convergencia más rápida que steepest descent
- ✅ No requiere almacenar el Hessiano
- ✅ Memoria O(N) en lugar de O(N²)
- ✅ Bueno para sistemas grandes

**Desventajas:**
- ❌ Convergencia cuadrática solo para funciones cuadráticas
- ❌ Puede perder la propiedad de conjugación con line search inexacto
- ❌ Más lento que métodos de segundo orden

**Cuándo usar:**
- Sistemas de tamaño medio (100-10,000 átomos)
- Cuando el cálculo del Hessiano es costoso
- Optimización de proteínas y sistemas biológicos

### 3. Newton-Raphson (Método de Newton)

**Principio:** Usa información de segunda derivada (curvatura) para convergencia cuadrática

**Aproximación cuadrática de Taylor:**
$$E(\mathbf{R} + \Delta\mathbf{R}) \approx E(\mathbf{R}) + \mathbf{g}^T \Delta\mathbf{R} + \frac{1}{2} \Delta\mathbf{R}^T \mathbf{H} \Delta\mathbf{R}$$

**Paso de Newton:**
$$\Delta\mathbf{R} = -\mathbf{H}^{-1} \mathbf{g}$$

$$\mathbf{R}_{n+1} = \mathbf{R}_n - \mathbf{H}_n^{-1} \mathbf{g}_n$$

**Ventajas:**
- ✅ Convergencia cuadrática cerca del mínimo
- ✅ Muy rápido cuando está cerca de la solución
- ✅ Usa información completa de curvatura

**Desventajas:**
- ❌ Requiere calcular e invertir el Hessiano (O(N³))
- ❌ Muy costoso para sistemas grandes
- ❌ Puede diverger si la geometría inicial es mala
- ❌ El Hessiano debe ser definido positivo

**Cuándo usar:**
- Moléculas pequeñas (<100 átomos)
- Refinamiento final cerca del mínimo
- Cálculos de alta precisión
- Cuando se necesitan frecuencias vibracionales

### 4. Quasi-Newton (BFGS, L-BFGS)

**Principio:** Aproximar el Hessiano iterativamente sin calcularlo explícitamente

**BFGS (Broyden-Fletcher-Goldfarb-Shanno):**
$$\mathbf{H}_{n+1}^{-1} = \mathbf{H}_n^{-1} + \frac{(\mathbf{s}_n - \mathbf{H}_n^{-1}\mathbf{y}_n)\mathbf{s}_n^T + \mathbf{s}_n(\mathbf{s}_n - \mathbf{H}_n^{-1}\mathbf{y}_n)^T}{\mathbf{s}_n^T\mathbf{y}_n} - \frac{(\mathbf{s}_n - \mathbf{H}_n^{-1}\mathbf{y}_n)^T\mathbf{y}_n}{\mathbf{s}_n^T\mathbf{y}_n} \frac{\mathbf{s}_n\mathbf{s}_n^T}{\mathbf{s}_n^T\mathbf{y}_n}$$

Donde:
- $\mathbf{s}_n = \mathbf{R}_{n+1} - \mathbf{R}_n$
- $\mathbf{y}_n = \mathbf{g}_{n+1} - \mathbf{g}_n$

**L-BFGS (Limited-memory BFGS):**
- Almacena solo los últimos $m$ vectores (típicamente $m = 3-20$)
- Memoria O(mN) en lugar de O(N²)
- Ideal para sistemas grandes

**Ventajas:**
- ✅ Convergencia superlineal
- ✅ No requiere calcular el Hessiano
- ✅ L-BFGS es muy eficiente para sistemas grandes
- ✅ Balance óptimo entre velocidad y memoria

**Desventajas:**
- ❌ Más complejo de implementar
- ❌ Requiere almacenar historia de pasos previos
- ❌ Puede perder convergencia si hay errores numéricos

**Cuándo usar:**
- **BFGS:** Sistemas pequeños-medianos (10-1000 átomos)
- **L-BFGS:** Sistemas grandes (>1000 átomos), optimización de proteínas
- **Default recomendado** para la mayoría de aplicaciones

### Comparación de Métodos Locales

| Método | Convergencia | Memoria | Costo/iteración | Robustez |
|--------|--------------|---------|-----------------|----------|
| Steepest Descent | Lineal | O(N) | Bajo | Alta |
| Conjugate Gradient | Superlineal | O(N) | Bajo | Media-Alta |
| Newton-Raphson | Cuadrática | O(N²) | Muy alto | Baja |
| BFGS | Superlineal | O(N²) | Medio | Alta |
| L-BFGS | Superlineal | O(mN) | Medio | Alta |

### Line Search (Búsqueda de Línea)

Todos los métodos anteriores requieren determinar el tamaño de paso óptimo $\alpha$ en cada iteración.

**Condición de Armijo (suficiente reducción):**
$$E(\mathbf{R}_n + \alpha\mathbf{d}_n) \leq E(\mathbf{R}_n) + c_1 \alpha \mathbf{g}_n^T \mathbf{d}_n$$

**Condición de Wolfe (curvatura):**
$$\mathbf{g}(\mathbf{R}_n + \alpha\mathbf{d}_n)^T \mathbf{d}_n \geq c_2 \mathbf{g}_n^T \mathbf{d}_n$$

Típicamente: $c_1 = 10^{-4}$, $c_2 = 0.9$

## 🎲 Métodos de Optimización Global

Los métodos globales buscan encontrar el mínimo absoluto en toda la PES, no solo el mínimo local más cercano.

### 1. Monte Carlo (MC)

**Principio:** Búsqueda aleatoria con criterio de aceptación

**Algoritmo básico:**
1. Generar geometría aleatoria $\mathbf{R}_{\text{new}}$
2. Calcular $\Delta E = E(\mathbf{R}_{\text{new}}) - E(\mathbf{R}_{\text{old}})$
3. Aceptar si $\Delta E < 0$ (siempre acepta mejoras)
4. Si $\Delta E > 0$, aceptar con probabilidad $P = \exp(-\Delta E / k_B T)$

**Ventajas:**
- ✅ Puede escapar de mínimos locales
- ✅ Muy simple de implementar
- ✅ No requiere derivadas
- ✅ Paralelizable

**Desventajas:**
- ❌ Muy ineficiente para optimización
- ❌ Requiere millones de evaluaciones
- ❌ No usa información del gradiente

### 2. Simulated Annealing (Recocido Simulado)

**Principio:** Monte Carlo con temperatura decreciente

$$T(n) = T_0 \cdot \alpha^n \quad \text{o} \quad T(n) = \frac{T_0}{\log(1+n)}$$

**Esquema de enfriamiento:**
- **Lineal:** $T(n) = T_0 - \beta n$
- **Exponencial:** $T(n) = T_0 \alpha^n$ (típico: $\alpha = 0.85-0.95$)
- **Logarítmico:** $T(n) = T_0 / \log(n+1)$

**Algoritmo:**
```
T = T_0  (temperatura inicial alta)
Mientras T > T_min:
    Generar geometría perturbada
    ΔE = E_new - E_old
    Si ΔE < 0: aceptar
    Si ΔE > 0: aceptar con P = exp(-ΔE/kT)
    Reducir T según esquema de enfriamiento
```

**Ventajas:**
- ✅ Garantiza convergencia al mínimo global (límite teórico)
- ✅ Puede escapar de mínimos locales al inicio
- ✅ Aplicable a cualquier tipo de sistema

**Desventajas:**
- ❌ Convergencia extremadamente lenta
- ❌ Resultado depende del esquema de enfriamiento
- ❌ Difícil elegir parámetros óptimos

### 3. Basin Hopping

**Principio:** Combina MC con optimización local

**Algoritmo:**
1. Partir de geometría inicial $\mathbf{R}_0$
2. Optimizar localmente → obtener $\mathbf{R}_{\text{min}}$
3. Perturbar aleatoriamente → $\mathbf{R}_{\text{new}}$
4. Optimizar localmente → $\mathbf{R}_{\text{min,new}}$
5. Aplicar criterio Metropolis sobre energías mínimas
6. Repetir hasta convergencia

**Ventajas:**
- ✅ Mucho más eficiente que MC puro
- ✅ Transforma PES en cuencas (basins)
- ✅ Muy efectivo para clusters y conformaciones

**Desventajas:**
- ❌ Requiere muchas optimizaciones locales
- ❌ Costoso computacionalmente

### 4. Algoritmos Genéticos (GA)

**Principio:** Evolución darwiniana aplicada a geometrías moleculares

**Componentes:**
- **Población:** Conjunto de geometrías ($N_{\text{pop}} = 20-200$)
- **Fitness:** $f = -E$ (menor energía = mayor fitness)
- **Selección:** Ruleta, torneo, ranking
- **Cruce (crossover):** Combinar geometrías parentales
- **Mutación:** Modificación aleatoria (probabilidad $p_m = 0.01-0.1$)

**Operadores para geometrías moleculares:**

**Crossover de coordenadas:**
$$\mathbf{R}_{\text{hijo}} = w \mathbf{R}_{\text{padre1}} + (1-w) \mathbf{R}_{\text{padre2}}$$

**Mutación de coordenadas:**
$$\mathbf{R}_i' = \mathbf{R}_i + \sigma \mathcal{N}(0,1)$$

**Ventajas:**
- ✅ Muy robusto para mínimos globales
- ✅ Altamente paralelizable
- ✅ No requiere derivadas
- ✅ Maneja espacios discontinuos

**Desventajas:**
- ❌ Muchos parámetros a ajustar
- ❌ Costoso (muchas evaluaciones de energía)
- ❌ Convergencia lenta

### 5. Particle Swarm Optimization (PSO)

**Principio:** Simulación de comportamiento de enjambres

$$\mathbf{v}_i^{t+1} = w\mathbf{v}_i^t + c_1 r_1 (\mathbf{p}_i - \mathbf{R}_i^t) + c_2 r_2 (\mathbf{g} - \mathbf{R}_i^t)$$

$$\mathbf{R}_i^{t+1} = \mathbf{R}_i^t + \mathbf{v}_i^{t+1}$$

Donde:
- $\mathbf{p}_i$ = mejor posición personal
- $\mathbf{g}$ = mejor posición global
- $w$ = inercia (típico: 0.4-0.9)
- $c_1, c_2$ = coeficientes de aprendizaje (típico: 2.0)
- $r_1, r_2$ = números aleatorios [0,1]

**Ventajas:**
- ✅ Pocas parámetros a ajustar
- ✅ Fácil de implementar
- ✅ Convergencia relativamente rápida

**Desventajas:**
- ❌ Puede converger prematuramente
- ❌ Sensible a parámetros

### 6. Differential Evolution (DE)

**Principio:** Evolución usando diferencias vectoriales

**Algoritmo:**
```
Para cada individuo R_i en la población:
    Seleccionar aleatoriamente R_a, R_b, R_c
    Crear mutante: R_m = R_a + F*(R_b - R_c)
    Crear trial: R_t = crossover(R_i, R_m)
    Si E(R_t) < E(R_i): reemplazar R_i con R_t
```

Parámetros:
- $F$ = factor de escala (típico: 0.5-1.0)
- $CR$ = probabilidad de cruce (típico: 0.7-0.9)

**Ventajas:**
- ✅ Muy robusto
- ✅ Pocos parámetros
- ✅ Excelente para espacios continuos

**Desventajas:**
- ❌ Puede ser lento
- ❌ Requiere evaluaciones de energía

### Comparación de Métodos Globales

| Método | Eficiencia | Robustez | Paralelización | Parámetros |
|--------|------------|----------|----------------|------------|
| Monte Carlo | Muy baja | Baja | Excelente | Pocos |
| Simulated Annealing | Baja | Alta | Limitada | Muchos |
| Basin Hopping | Media | Alta | Buena | Pocos |
| Algoritmos Genéticos | Media | Muy alta | Excelente | Muchos |
| PSO | Media-Alta | Media | Excelente | Pocos |
| Differential Evolution | Alta | Muy alta | Excelente | Pocos |

## ⚙️ Criterios de Convergencia

### Criterios Basados en Gradiente

**RMS del gradiente:**
$$\text{RMS}(\mathbf{g}) = \sqrt{\frac{1}{3N} \sum_{i=1}^{3N} g_i^2} < \epsilon_g$$

Típico: $\epsilon_g = 10^{-4}$ a $10^{-6}$ hartree/bohr

**Máxima componente del gradiente:**
$$\max(|\mathbf{g}|) < \epsilon_{\max}$$

### Criterios Basados en Energía

**Cambio en energía:**
$$|\Delta E| = |E_n - E_{n-1}| < \epsilon_E$$

Típico: $\epsilon_E = 10^{-6}$ a $10^{-9}$ hartree

### Criterios Basados en Geometría

**RMS del desplazamiento:**
$$\text{RMS}(\Delta\mathbf{R}) = \sqrt{\frac{1}{3N} \sum_{i=1}^{3N} \Delta R_i^2} < \epsilon_R$$

Típico: $\epsilon_R = 10^{-4}$ Å

**Máximo desplazamiento:**
$$\max(|\Delta\mathbf{R}|) < \epsilon_{\max,R}$$

### Criterios Estándar por Software

**Gaussian (criterio por defecto):**
- Max Force < 0.000450 hartree/bohr
- RMS Force < 0.000300 hartree/bohr
- Max Displacement < 0.001800 bohr
- RMS Displacement < 0.001200 bohr

**ORCA (TightOPT):**
- Max gradient < 0.0001 hartree/bohr
- RMS gradient < 0.00003 hartree/bohr
- Max displacement < 0.001 bohr
- RMS displacement < 0.0005 bohr

**Recomendación:** Usar criterios múltiples simultáneamente para asegurar convergencia verdadera.

## 🔧 Configuración del Entorno

In [ ]:
# Verificar si estamos en Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Ejecutando en Google Colab")
    print("📦 Instalando paquetes necesarios...")
    
    # Instalar RDKit
    !pip install -q rdkit
    
    # Instalar py3Dmol para visualización
    !pip install -q py3Dmol
    
    # Instalar ASE (Atomic Simulation Environment)
    !pip install -q ase
    
    # Instalar scipy para optimización
    !pip install -q scipy
    
    print("✓ Paquetes instalados correctamente\n")
else:
    print("✓ Ejecutando localmente")
    print("⚠️  Asegúrate de tener instalados: rdkit, py3Dmol, ase, scipy")

In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import warnings
import time
warnings.filterwarnings('ignore')

# RDKit para química computacional
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors, rdMolTransforms
from rdkit.Chem.Draw import IPythonConsole

# Visualización 3D
import py3Dmol

# Scipy para optimización
from scipy.optimize import minimize, differential_evolution

# ASE para cálculos moleculares
try:
    from ase import Atoms
    from ase.optimize import BFGS, LBFGS, BFGSLineSearch
    ASE_AVAILABLE = True
except:
    ASE_AVAILABLE = False
    print("⚠️ ASE no disponible")

# Configuración de matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Librerías importadas correctamente")
print(f"✓ Versión de RDKit: {Chem.__version__}")
print(f"✓ ASE disponible: {ASE_AVAILABLE}")

## 🧪 Parte 1: Implementación de Métodos de Optimización

### Caso de Estudio: Función de Rosenbrock (Ejemplo 2D)

Antes de aplicar a moléculas, vamos a visualizar cómo funcionan los métodos de optimización en una función simple.

La **función de Rosenbrock** es un benchmark clásico:

$$f(x, y) = (1-x)^2 + 100(y-x^2)^2$$

Mínimo global en $(x^*, y^*) = (1, 1)$ con $f(1, 1) = 0$

In [ ]:
# Definir la función de Rosenbrock y su gradiente
def rosenbrock(X):
    """Función de Rosenbrock"""
    x, y = X
    return (1 - x)**2 + 100*(y - x**2)**2

def rosenbrock_grad(X):
    """Gradiente de la función de Rosenbrock"""
    x, y = X
    dfdx = -2*(1 - x) - 400*x*(y - x**2)
    dfdy = 200*(y - x**2)
    return np.array([dfdx, dfdy])

# Crear superficie para visualización
x = np.linspace(-2, 2, 100)
y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100*(Y - X**2)**2

# Visualizar la superficie
fig = plt.figure(figsize=(15, 5))

# Vista 3D
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.6, edgecolor='none')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('f(x,y)')
ax1.set_title('Función de Rosenbrock - Vista 3D')
ax1.view_init(elev=30, azim=45)

# Contornos
ax2 = fig.add_subplot(132)
contours = ax2.contour(X, Y, Z, levels=np.logspace(-1, 3.5, 20), cmap='viridis')
ax2.clabel(contours, inline=True, fontsize=8)
ax2.plot(1, 1, 'r*', markersize=15, label='Mínimo global')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('Contornos')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Log-escala
ax3 = fig.add_subplot(133)
contours = ax3.contour(X, Y, np.log10(Z + 1), levels=20, cmap='viridis')
ax3.plot(1, 1, 'r*', markersize=15, label='Mínimo global')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_title('Contornos (log-escala)')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mínimo en (1, 1), f(1,1) = {rosenbrock([1, 1]):.6f}")

### Implementación de Steepest Descent

In [ ]:
def steepest_descent(func, grad_func, x0, alpha=0.001, max_iter=1000, tol=1e-6):
    """
    Método Steepest Descent
    
    Args:
        func: Función objetivo
        grad_func: Función gradiente
        x0: Punto inicial
        alpha: Tamaño de paso
        max_iter: Máximo de iteraciones
        tol: Tolerancia de convergencia
    
    Returns:
        x_opt: Solución óptima
        history: Historia de la optimización
    """
    x = np.array(x0, dtype=float)
    history = {'x': [x.copy()], 'f': [func(x)], 'grad_norm': [np.linalg.norm(grad_func(x))]}
    
    for i in range(max_iter):
        grad = grad_func(x)
        grad_norm = np.linalg.norm(grad)
        
        # Criterio de convergencia
        if grad_norm < tol:
            print(f"✓ Convergió en {i+1} iteraciones")
            break
        
        # Actualizar posición
        x = x - alpha * grad
        
        # Guardar historia
        history['x'].append(x.copy())
        history['f'].append(func(x))
        history['grad_norm'].append(grad_norm)
    else:
        print(f"⚠️ No convergió en {max_iter} iteraciones")
    
    return x, history

# Probar Steepest Descent
x0 = np.array([-1.5, 2.5])
print(f"Punto inicial: {x0}, f(x0) = {rosenbrock(x0):.4f}")
print("\nOptimizando con Steepest Descent...")

x_opt_sd, history_sd = steepest_descent(rosenbrock, rosenbrock_grad, x0, 
                                        alpha=0.001, max_iter=10000)

print(f"\nSolución: {x_opt_sd}")
print(f"f(x_opt) = {rosenbrock(x_opt_sd):.6f}")
print(f"Iteraciones: {len(history_sd['x'])}")
print(f"Gradiente final: {np.linalg.norm(rosenbrock_grad(x_opt_sd)):.2e}")

### Comparación de Métodos de Optimización

In [ ]:
# Comparar diferentes métodos usando scipy.optimize
from scipy.optimize import minimize

x0 = np.array([-1.5, 2.5])
metodos = ['Nelder-Mead', 'CG', 'BFGS', 'L-BFGS-B']

resultados = {}

print("🔬 Comparando métodos de optimización\n")
print("="*70)

for metodo in metodos:
    start_time = time.time()
    
    if metodo in ['CG', 'BFGS', 'L-BFGS-B']:
        result = minimize(rosenbrock, x0, method=metodo, jac=rosenbrock_grad, 
                         options={'disp': False})
    else:
        result = minimize(rosenbrock, x0, method=metodo, options={'disp': False})
    
    tiempo = time.time() - start_time
    
    resultados[metodo] = {
        'x': result.x,
        'f': result.fun,
        'nfev': result.nfev,
        'nit': result.nit,
        'tiempo': tiempo,
        'exito': result.success
    }
    
    print(f"\n{metodo}:")
    print(f"  Solución: [{result.x[0]:.6f}, {result.x[1]:.6f}]")
    print(f"  f(x) = {result.fun:.2e}")
    print(f"  Iteraciones: {result.nit}")
    print(f"  Evaluaciones: {result.nfev}")
    print(f"  Tiempo: {tiempo*1000:.2f} ms")
    print(f"  Éxito: {'✓' if result.success else '✗'}")

print("\n" + "="*70)

In [ ]:
# Visualizar trayectorias de optimización
fig, ax = plt.subplots(figsize=(10, 8))

# Contornos de fondo
contours = ax.contour(X, Y, np.log10(Z + 1), levels=20, cmap='gray', alpha=0.3)

# Graficar trayectoria de Steepest Descent
path_sd = np.array(history_sd['x'])
ax.plot(path_sd[:, 0], path_sd[:, 1], 'o-', label='Steepest Descent', 
        linewidth=2, markersize=3, alpha=0.7)

# Punto inicial y final
ax.plot(x0[0], x0[1], 'go', markersize=12, label='Inicio', zorder=5)
ax.plot(1, 1, 'r*', markersize=20, label='Mínimo global', zorder=5)

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Trayectoria de Optimización - Steepest Descent', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(-2, 2)
ax.set_ylim(-1, 3)

plt.tight_layout()
plt.show()

# Graficar convergencia
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Energía vs iteración
axes[0].semilogy(history_sd['f'], 'b-', linewidth=2)
axes[0].set_xlabel('Iteración', fontsize=12)
axes[0].set_ylabel('f(x) [escala log]', fontsize=12)
axes[0].set_title('Convergencia de Energía', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Norma del gradiente
axes[1].semilogy(history_sd['grad_norm'], 'r-', linewidth=2)
axes[1].axhline(y=1e-6, color='green', linestyle='--', label='Tolerancia')
axes[1].set_xlabel('Iteración', fontsize=12)
axes[1].set_ylabel('||∇f|| [escala log]', fontsize=12)
axes[1].set_title('Convergencia del Gradiente', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 💊 Parte 2: Optimización de Moléculas Simples

### Caso 1: Optimización de Agua (H₂O)

In [ ]:
# Crear molécula de agua con geometría distorsionada
smiles = "O"
mol_water = Chem.MolFromSmiles(smiles)
mol_water = Chem.AddHs(mol_water)

# Generar geometría 3D inicial (no optimizada)
AllChem.EmbedMolecule(mol_water, randomSeed=42)

print("💧 Molécula de Agua")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_water)}")
print(f"  Número de átomos: {mol_water.GetNumAtoms()}")

# Visualizar estructura inicial
Draw.MolToImage(mol_water, size=(300, 300))

In [ ]:
# Función para visualizar moléculas en 3D
def visualizar_molecula_3d(mol, width=400, height=400, style='stick'):
    """Visualiza una molécula en 3D usando py3Dmol"""
    viewer = py3Dmol.view(width=width, height=height)
    mol_block = Chem.MolToMolBlock(mol)
    viewer.addModel(mol_block, 'mol')
    viewer.setStyle({style: {}})
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    return viewer

print("Geometría inicial (no optimizada):")
visualizar_molecula_3d(mol_water)

In [ ]:
# Función para extraer geometría y calcular parámetros
def obtener_parametros_agua(mol):
    """Extrae parámetros geométricos del agua"""
    conf = mol.GetConformer()
    
    # Índices: O=0, H=1, H=2
    # Distancias O-H
    d_OH1 = rdMolTransforms.GetBondLength(conf, 0, 1)
    d_OH2 = rdMolTransforms.GetBondLength(conf, 0, 2)
    
    # Ángulo H-O-H
    angle_HOH = rdMolTransforms.GetAngleDeg(conf, 1, 0, 2)
    
    return d_OH1, d_OH2, angle_HOH

# Geometría inicial
d1_i, d2_i, ang_i = obtener_parametros_agua(mol_water)
print(f"\nGeometría inicial:")
print(f"  d(O-H1) = {d1_i:.4f} Å")
print(f"  d(O-H2) = {d2_i:.4f} Å")
print(f"  ∠(H-O-H) = {ang_i:.2f}°")

### Optimización con Diferentes Métodos

In [ ]:
# Comparar diferentes métodos de optimización

# Crear copias de la molécula
mol_mmff_sd = Chem.Mol(mol_water)
mol_mmff_cg = Chem.Mol(mol_water)
mol_uff = Chem.Mol(mol_water)

metodos_opt = {}

print("🔬 Optimizando agua con diferentes configuraciones\n")
print("="*70)

# 1. MMFF94 con configuración por defecto
print("\n1. MMFF94 (configuración por defecto):")
start = time.time()
result_mmff = AllChem.MMFFOptimizeMolecule(mol_mmff_sd, maxIters=500)
tiempo_mmff = time.time() - start

props_mmff = AllChem.MMFFGetMoleculeProperties(mol_mmff_sd)
ff_mmff = AllChem.MMFFGetMoleculeForceField(mol_mmff_sd, props_mmff)
energia_mmff = ff_mmff.CalcEnergy()

d1_mmff, d2_mmff, ang_mmff = obtener_parametros_agua(mol_mmff_sd)

print(f"  Convergió: {'✓' if result_mmff == 0 else '✗'}")
print(f"  Energía: {energia_mmff:.6f} kcal/mol")
print(f"  d(O-H) promedio: {(d1_mmff + d2_mmff)/2:.4f} Å")
print(f"  ∠(H-O-H): {ang_mmff:.2f}°")
print(f"  Tiempo: {tiempo_mmff*1000:.2f} ms")

metodos_opt['MMFF94'] = {
    'mol': mol_mmff_sd,
    'energia': energia_mmff,
    'd_OH': (d1_mmff + d2_mmff)/2,
    'angulo': ang_mmff,
    'tiempo': tiempo_mmff
}

# 2. UFF
print("\n2. UFF:")
start = time.time()
result_uff = AllChem.UFFOptimizeMolecule(mol_uff, maxIters=500)
tiempo_uff = time.time() - start

ff_uff = AllChem.UFFGetMoleculeForceField(mol_uff)
energia_uff = ff_uff.CalcEnergy()

d1_uff, d2_uff, ang_uff = obtener_parametros_agua(mol_uff)

print(f"  Convergió: {'✓' if result_uff == 0 else '✗'}")
print(f"  Energía: {energia_uff:.6f} kcal/mol")
print(f"  d(O-H) promedio: {(d1_uff + d2_uff)/2:.4f} Å")
print(f"  ∠(H-O-H): {ang_uff:.2f}°")
print(f"  Tiempo: {tiempo_uff*1000:.2f} ms")

metodos_opt['UFF'] = {
    'mol': mol_uff,
    'energia': energia_uff,
    'd_OH': (d1_uff + d2_uff)/2,
    'angulo': ang_uff,
    'tiempo': tiempo_uff
}

print("\n" + "="*70)
print("\n💡 Valores experimentales de referencia:")
print("  d(O-H) = 0.9572 Å")
print("  ∠(H-O-H) = 104.52°")

In [ ]:
# Tabla comparativa
data_comparacion = {
    'Método': ['MMFF94', 'UFF', 'Experimental'],
    'd(O-H) (Å)': [
        metodos_opt['MMFF94']['d_OH'],
        metodos_opt['UFF']['d_OH'],
        0.9572
    ],
    '∠(H-O-H) (°)': [
        metodos_opt['MMFF94']['angulo'],
        metodos_opt['UFF']['angulo'],
        104.52
    ],
    'Energía (kcal/mol)': [
        metodos_opt['MMFF94']['energia'],
        metodos_opt['UFF']['energia'],
        np.nan
    ],
    'Tiempo (ms)': [
        metodos_opt['MMFF94']['tiempo']*1000,
        metodos_opt['UFF']['tiempo']*1000,
        np.nan
    ]
}

df_comp = pd.DataFrame(data_comparacion)
print("\n📊 Tabla Comparativa:")
print(df_comp.to_string(index=False))

# Calcular errores
print("\n📏 Errores respecto a valores experimentales:")
for metodo in ['MMFF94', 'UFF']:
    error_d = abs(metodos_opt[metodo]['d_OH'] - 0.9572)
    error_ang = abs(metodos_opt[metodo]['angulo'] - 104.52)
    print(f"\n{metodo}:")
    print(f"  Error en d(O-H): {error_d:.4f} Å ({error_d/0.9572*100:.2f}%)")
    print(f"  Error en ∠(H-O-H): {error_ang:.2f}° ({error_ang/104.52*100:.2f}%)")

## 🧬 Parte 3: Optimización de Moléculas Orgánicas

### Caso 2: n-Pentano - Múltiples Conformaciones

In [ ]:
# Crear n-pentano
smiles_pentano = "CCCCC"
mol_pentano = Chem.MolFromSmiles(smiles_pentano)
mol_pentano = Chem.AddHs(mol_pentano)

print("🔬 n-Pentano")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_pentano)}")
print(f"  Número de átomos: {mol_pentano.GetNumAtoms()}")
print(f"  Enlaces rotables: {Descriptors.NumRotatableBonds(mol_pentano)}")

# Visualizar 2D
Draw.MolToImage(mol_pentano, size=(400, 200))

In [ ]:
# Generar múltiples conformaciones y optimizarlas
print("🔄 Generando y optimizando conformaciones de n-pentano...\n")

# Generar conformaciones iniciales
num_conf = 50
conf_ids = AllChem.EmbedMultipleConfs(mol_pentano, numConfs=num_conf, 
                                      randomSeed=42, clearConfs=True)

print(f"✓ Se generaron {len(conf_ids)} conformaciones")

# Optimizar cada conformación con MMFF94
energias = []
for conf_id in conf_ids:
    # Optimizar
    AllChem.MMFFOptimizeMolecule(mol_pentano, confId=conf_id, maxIters=500)
    
    # Calcular energía
    props = AllChem.MMFFGetMoleculeProperties(mol_pentano)
    ff = AllChem.MMFFGetMoleculeForceField(mol_pentano, props, confId=conf_id)
    energia = ff.CalcEnergy()
    energias.append(energia)

energias = np.array(energias)
energias_rel = energias - energias.min()

# Encontrar conformaciones únicas de baja energía
idx_sorted = np.argsort(energias)
conformaciones_bajas = idx_sorted[:10]

print(f"\n📊 Top 10 conformaciones de menor energía:")
print("="*50)
for i, idx in enumerate(conformaciones_bajas, 1):
    print(f"  {i}. Conformación {idx}: {energias_rel[idx]:.3f} kcal/mol")

In [ ]:
# Histograma de energías conformacionales
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(energias_rel, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='green', linestyle='--', linewidth=2, 
                label='Conformación más estable')
axes[0].set_xlabel('Energía Relativa (kcal/mol)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
axes[0].set_title('Distribución de Energías Conformacionales', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Scatter plot
axes[1].scatter(range(len(energias_rel)), energias_rel, alpha=0.6, s=50)
axes[1].scatter(conformaciones_bajas, energias_rel[conformaciones_bajas], 
                color='red', s=100, label='Top 10', zorder=5)
axes[1].set_xlabel('Índice de Conformación', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Energía Relativa (kcal/mol)', fontsize=12, fontweight='bold')
axes[1].set_title('Energías de Todas las Conformaciones', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Rango de energías: {energias_rel.min():.3f} - {energias_rel.max():.3f} kcal/mol")
print(f"   Diferencia: {energias_rel.max():.3f} kcal/mol")

### Visualización de las Conformaciones más Estables

In [ ]:
# Visualizar las 3 conformaciones más estables
print("🔍 Las 3 conformaciones más estables:\n")

for i in range(min(3, len(conformaciones_bajas))):
    idx = conformaciones_bajas[i]
    energia = energias_rel[idx]
    
    # Crear molécula temporal con esa conformación
    mol_temp = Chem.Mol(mol_pentano)
    mol_temp.RemoveAllConformers()
    mol_temp.AddConformer(mol_pentano.GetConformer(idx))
    
    print(f"Conformación {i+1} (#{idx}): {energia:.3f} kcal/mol")
    display(visualizar_molecula_3d(mol_temp, width=400, height=300))

## 💊 Parte 4: Fármacos - Caso de Aspirina

### Optimización Multi-inicio

In [ ]:
# Crear aspirina (ácido acetilsalicílico)
smiles_aspirina = "CC(=O)Oc1ccccc1C(=O)O"
mol_aspirina = Chem.MolFromSmiles(smiles_aspirina)
mol_aspirina = Chem.AddHs(mol_aspirina)

print("💊 Aspirina (Ácido Acetilsalicílico)")
print(f"  Fórmula: {Chem.rdMolDescriptors.CalcMolFormula(mol_aspirina)}")
print(f"  Peso Molecular: {Descriptors.MolWt(mol_aspirina):.2f} g/mol")
print(f"  Número de átomos: {mol_aspirina.GetNumAtoms()}")
print(f"  Enlaces rotables: {Descriptors.NumRotatableBonds(mol_aspirina)}")

# Visualizar 2D
Draw.MolToImage(mol_aspirina, size=(400, 300))

In [ ]:
# Estrategia: Múltiples optimizaciones desde diferentes geometrías iniciales
print("🔬 Optimización multi-inicio de aspirina\n")
print("="*70)

num_inicios = 30
energias_aspirina = []
convergencias = []

for i in range(num_inicios):
    # Crear copia
    mol_temp = Chem.Mol(mol_aspirina)
    
    # Generar geometría inicial aleatoria
    AllChem.EmbedMolecule(mol_temp, randomSeed=42+i)
    
    # Optimizar con MMFF94
    resultado = AllChem.MMFFOptimizeMolecule(mol_temp, maxIters=1000)
    
    # Calcular energía
    props = AllChem.MMFFGetMoleculeProperties(mol_temp)
    ff = AllChem.MMFFGetMoleculeForceField(mol_temp, props)
    energia = ff.CalcEnergy()
    
    energias_aspirina.append(energia)
    convergencias.append(resultado == 0)
    
    if (i+1) % 10 == 0:
        print(f"  Completado {i+1}/{num_inicios} optimizaciones...")

energias_aspirina = np.array(energias_aspirina)
energia_min = energias_aspirina.min()
energias_rel_asp = energias_aspirina - energia_min

print(f"\n✓ Optimizaciones completadas")
print(f"  Convergieron: {sum(convergencias)}/{num_inicios}")
print(f"  Energía mínima: {energia_min:.4f} kcal/mol")
print(f"  Energía máxima: {energias_aspirina.max():.4f} kcal/mol")
print(f"  Rango: {energias_rel_asp.max():.4f} kcal/mol")

In [ ]:
# Análisis estadístico
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(energias_rel_asp, bins=15, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='green', linestyle='--', linewidth=2, 
                label=f'Mínimo global')
axes[0].axvline(x=energias_rel_asp.mean(), color='blue', linestyle='--', linewidth=2, 
                label=f'Media: {energias_rel_asp.mean():.2f} kcal/mol')
axes[0].set_xlabel('Energía Relativa (kcal/mol)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
axes[0].set_title('Distribución de Mínimos Encontrados - Aspirina', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(energias_rel_asp, vert=True, patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[1].set_ylabel('Energía Relativa (kcal/mol)', fontsize=12, fontweight='bold')
axes[1].set_title('Estadística de Energías', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Estadísticas
print(f"\n📊 Estadísticas:")
print(f"  Media: {energias_rel_asp.mean():.3f} kcal/mol")
print(f"  Mediana: {np.median(energias_rel_asp):.3f} kcal/mol")
print(f"  Desv. estándar: {energias_rel_asp.std():.3f} kcal/mol")
print(f"  Cuartil 25%: {np.percentile(energias_rel_asp, 25):.3f} kcal/mol")
print(f"  Cuartil 75%: {np.percentile(energias_rel_asp, 75):.3f} kcal/mol")

## 🎯 Parte 5: Ejercicios Prácticos

### Ejercicio 1: Optimización de Metanol

Optimiza el metanol (CH₃OH) con MMFF94 y UFF, compara los resultados con valores experimentales.

In [ ]:
# EJERCICIO: Optimizar metanol
# Valores experimentales de referencia:
# d(C-O) ≈ 1.43 Å
# d(O-H) ≈ 0.96 Å
# ∠(C-O-H) ≈ 108.5°

smiles_metanol = "CO"

# TU CÓDIGO AQUÍ:
# 1. Crear la molécula
# 2. Generar geometría 3D
# 3. Optimizar con MMFF94 y UFF
# 4. Extraer distancias de enlace y ángulos
# 5. Comparar con valores experimentales
# 6. Calcular errores porcentuales

print("🔬 Ejercicio: Optimización de Metanol\n")

# Plantilla:
mol_metanol = Chem.MolFromSmiles(smiles_metanol)
mol_metanol = Chem.AddHs(mol_metanol)

# Continúa aquí...

### Ejercicio 2: Conformaciones de Etano

Genera el perfil de energía del etano rotando el enlace C-C con diferentes métodos de optimización.

In [ ]:
# EJERCICIO: Perfil conformacional del etano
# Compara la barrera de rotación usando diferentes optimizadores

smiles_etano = "CC"

# TU CÓDIGO AQUÍ:
# 1. Crear etano
# 2. Para cada ángulo diedro (0° a 360°, paso 10°):
#    a. Fijar el ángulo H-C-C-H
#    b. Optimizar manteniendo el diedro fijo (constraint optimization)
#    c. Calcular energía
# 3. Graficar perfil de energía
# 4. Identificar conformaciones eclipsada (0°) y escalonada (60°)
# 5. Calcular barrera de rotación
# 6. Comparar con valor experimental (~3 kcal/mol)

### Ejercicio 3: Cicloalcanos - Tensión Anular

Optimiza ciclopropano, ciclobutano, ciclopentano y ciclohexano. Calcula la energía de tensión por átomo de carbono.

In [ ]:
# EJERCICIO: Tensión anular en cicloalcanos

cicloalcanos = {
    'Ciclopropano': 'C1CC1',
    'Ciclobutano': 'C1CCC1',
    'Ciclopentano': 'C1CCCC1',
    'Ciclohexano': 'C1CCCCC1'
}

# TU CÓDIGO AQUÍ:
# 1. Optimizar cada cicloalcano con MMFF94
# 2. Calcular energía total y energía por átomo de carbono
# 3. Calcular ángulos de enlace C-C-C
# 4. Comparar con ángulos ideales:
#    - Ciclopropano: 60° (vs 109.5° ideal → alta tensión)
#    - Ciclobutano: 90°
#    - Ciclopentano: 108°
#    - Ciclohexano: ~109.5° (sin tensión)
# 5. Graficar energía vs tamaño de anillo
# 6. Relacionar tensión con desviación del ángulo tetraédrico

print("🔬 Ejercicio: Tensión Anular en Cicloalcanos\n")

# Tu implementación aquí...

### Ejercicio 4: Isómeros de Butano

Optimiza n-butano e iso-butano. Compara sus energías y propiedades geométricas.

In [ ]:
# EJERCICIO: Comparación de isómeros

isomeros_butano = {
    'n-Butano': 'CCCC',
    'iso-Butano': 'CC(C)C'
}

# TU CÓDIGO AQUÍ:
# 1. Optimizar ambos isómeros
# 2. Comparar energías (¿cuál es más estable?)
# 3. Analizar distancias de enlace C-C
# 4. Analizar ángulos de enlace
# 5. Explicar diferencias estructurales
# 6. Relacionar con estabilidad termodinámica

print("🔬 Ejercicio: Isómeros de Butano\n")

# Tu código aquí...

### Ejercicio 5: Búsqueda de Conformaciones de Glucosa

La glucosa tiene múltiples conformaciones. Realiza una búsqueda sistemática.

In [ ]:
# EJERCICIO AVANZADO: Conformaciones de α-D-glucosa

smiles_glucosa = "C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)O)O)O)O"

# TU CÓDIGO AQUÍ:
# 1. Generar 100+ conformaciones con EmbedMultipleConfs
# 2. Optimizar cada una con MMFF94
# 3. Agrupar por rangos de energía (<1 kcal/mol, 1-3, 3-5, >5)
# 4. Identificar conformaciones de silla y bote del anillo
# 5. Visualizar las 5 conformaciones más estables
# 6. Analizar la flexibilidad conformacional

print("🔬 Ejercicio Avanzado: Conformaciones de Glucosa\n")

# Pistas:
# - Usa un número alto de conformaciones iniciales
# - El anillo de 6 miembros preferirá conformación silla
# - Los grupos OH pueden rotar, generando muchas conformaciones
# - Considera usar restricciones geométricas si es necesario

## 📊 Parte 6: Mejores Prácticas y Problemas Comunes

### Problemas Comunes en Optimización

### 🚫 Problema 1: Geometría Inicial Mala

**Síntoma:** El optimizador no converge o converge a geometrías absurdas

**Causas:**
- Átomos solapados
- Distancias de enlace muy distorsionadas
- Quiralidad incorrecta
- Conformación imposible

**Soluciones:**
1. Usar generadores de geometría robustos (RDKit ETKDG)
2. Pre-optimización con métodos rápidos (steepest descent + UFF)
3. Verificar conectividad antes de optimizar
4. Usar librerías de fragmentos moleculares

### 🚫 Problema 2: Convergencia Lenta

**Síntoma:** Muchas iteraciones sin llegar al mínimo

**Causas:**
- Superficie muy plana
- Múltiples grados de libertad
- Método inadecuado (steepest descent en valles estrechos)

**Soluciones:**
1. Cambiar a método más eficiente (L-BFGS)
2. Ajustar tolerancias de convergencia
3. Optimización por etapas (congelar átomos)
4. Usar coordenadas internas en lugar de cartesianas

### 🚫 Problema 3: Mínimo Local Incorrecto

**Síntoma:** Energía final más alta que esperada, geometría extraña

**Causas:**
- Superficie de energía con múltiples mínimos
- Punto de partida en cuenca incorrecta
- Barreras conformacionales

**Soluciones:**
1. Múltiples optimizaciones desde geometrías diferentes
2. Usar métodos de optimización global (basin hopping)
3. Análisis de conformaciones sistemático
4. Simulated annealing para escapar mínimos locales

### 🚫 Problema 4: Pérdida de Simetría

**Síntoma:** Molécula simétrica converge a estructura asimétrica

**Causas:**
- Errores numéricos
- Tolerancias de convergencia muy laxas
- Punto de silla muy cercano al mínimo

**Soluciones:**
1. Usar restricciones de simetría
2. Mejorar tolerancias de convergencia
3. Seguimiento de modos de frecuencia imaginarios
4. Re-optimizar desde geometría simétrica

### ✅ Mejores Prácticas

1. **Siempre validar la geometría inicial**
   ```python
   # Verificar que la molécula es válida
   Chem.SanitizeMol(mol)
   # Verificar que tiene 3D
   assert mol.GetNumConformers() > 0
   ```

2. **Usar múltiples intentos para moléculas flexibles**
   ```python
   num_confs = 50  # Para moléculas con varios enlaces rotables
   conf_ids = AllChem.EmbedMultipleConfs(mol, numConfs=num_confs)
   ```

3. **Optimización en dos etapas**
   ```python
   # Etapa 1: Pre-optimización rápida
   AllChem.UFFOptimizeMolecule(mol, maxIters=200)
   # Etapa 2: Optimización fina
   AllChem.MMFFOptimizeMolecule(mol, maxIters=1000)
   ```

4. **Verificar convergencia**
   ```python
   result = AllChem.MMFFOptimizeMolecule(mol)
   if result != 0:
       print("⚠️ No convergió, intentar con más iteraciones")
   ```

5. **Comparar con datos experimentales cuando sea posible**
   - Distancias de enlace de cristalografía
   - Ángulos de RMN
   - Energías conformacionales de literatura

6. **Documentar parámetros utilizados**
   - Campo de fuerza
   - Método de optimización
   - Criterios de convergencia
   - Número de iteraciones

## 📈 Parte 7: Análisis Avanzado - Superficies de Energía Potencial 2D

### Scan 2D: Rotación de dos enlaces diedros

In [ ]:
# Ejemplo: Scan 2D de n-butano (rotación de dos diedros)
print("🔬 Scan 2D de n-butano\n")
print("⏳ Esto puede tomar varios minutos...\n")

# Crear butano
smiles_butano = "CCCC"
mol_butano_2d = Chem.MolFromSmiles(smiles_butano)
mol_butano_2d = Chem.AddHs(mol_butano_2d)
AllChem.EmbedMolecule(mol_butano_2d, randomSeed=42)

# Definir rangos de ángulos diedros
angulos = np.arange(0, 360, 30)  # Paso de 30° para rapidez
n_angulos = len(angulos)

# Matriz de energías
energia_2d = np.zeros((n_angulos, n_angulos))

# Realizar scan (esto es computacionalmente costoso)
print(f"Calculando {n_angulos}x{n_angulos} = {n_angulos**2} puntos...")

for i, ang1 in enumerate(angulos):
    for j, ang2 in enumerate(angulos):
        # Crear copia
        mol_temp = Chem.Mol(mol_butano_2d)
        conf = mol_temp.GetConformer()
        
        # Establecer ángulos diedros
        # Primer diedro: átomos 0-1-2-3
        rdMolTransforms.SetDihedralDeg(conf, 0, 1, 2, 3, float(ang1))
        
        # Calcular energía sin optimizar completamente
        # (solo evaluar energía en la geometría fijada)
        props = AllChem.MMFFGetMoleculeProperties(mol_temp)
        ff = AllChem.MMFFGetMoleculeForceField(mol_temp, props)
        energia_2d[i, j] = ff.CalcEnergy()
    
    if (i+1) % 3 == 0:
        print(f"  Completado {i+1}/{n_angulos} filas...")

print("✓ Scan completado\n")

# Normalizar energías
energia_2d_rel = energia_2d - energia_2d.min()

In [ ]:
# Visualizar PES 2D
fig = plt.figure(figsize=(16, 5))

# Superficie 3D
ax1 = fig.add_subplot(131, projection='3d')
X, Y = np.meshgrid(angulos, angulos)
surf = ax1.plot_surface(X, Y, energia_2d_rel, cmap='viridis', alpha=0.8)
ax1.set_xlabel('Diedro 1 (°)', fontsize=10)
ax1.set_ylabel('Diedro 2 (°)', fontsize=10)
ax1.set_zlabel('Energía (kcal/mol)', fontsize=10)
ax1.set_title('PES 2D - Vista 3D', fontsize=12, fontweight='bold')
fig.colorbar(surf, ax=ax1, shrink=0.5)

# Mapa de contornos
ax2 = fig.add_subplot(132)
contourf = ax2.contourf(X, Y, energia_2d_rel, levels=15, cmap='viridis')
contour = ax2.contour(X, Y, energia_2d_rel, levels=10, colors='white', 
                      linewidths=0.5, alpha=0.5)
ax2.set_xlabel('Diedro 1 (°)', fontsize=11)
ax2.set_ylabel('Diedro 2 (°)', fontsize=11)
ax2.set_title('Mapa de Contornos', fontsize=12, fontweight='bold')
fig.colorbar(contourf, ax=ax2, label='Energía (kcal/mol)')

# Mapa de calor
ax3 = fig.add_subplot(133)
im = ax3.imshow(energia_2d_rel, extent=[0, 360, 0, 360], origin='lower', 
                cmap='hot', aspect='auto')
ax3.set_xlabel('Diedro 1 (°)', fontsize=11)
ax3.set_ylabel('Diedro 2 (°)', fontsize=11)
ax3.set_title('Mapa de Calor', fontsize=12, fontweight='bold')
fig.colorbar(im, ax=ax3, label='Energía (kcal/mol)')

plt.tight_layout()
plt.show()

# Identificar mínimos
min_idx = np.unravel_index(energia_2d_rel.argmin(), energia_2d_rel.shape)
print(f"💡 Mínimo global encontrado en:")
print(f"   Diedro 1: {angulos[min_idx[0]]}°")
print(f"   Diedro 2: {angulos[min_idx[1]]}°")
print(f"   Energía: {energia_2d_rel[min_idx]:.3f} kcal/mol")

## 📚 Resumen y Conclusiones

### Puntos Clave

1. **La optimización de geometría busca mínimos en la PES**
   - Condición necesaria: gradiente = 0
   - Condición suficiente: Hessiano definido positivo

2. **Métodos locales vs globales**
   - Locales: rápidos, encuentran el mínimo más cercano
   - Globales: lentos, pueden encontrar el mínimo absoluto

3. **Jerarquía de métodos locales**
   - Steepest Descent: simple pero lento
   - Conjugate Gradient: buen balance
   - **L-BFGS: recomendado para la mayoría de casos**
   - Newton-Raphson: rápido cerca del mínimo pero costoso

4. **Criterios de convergencia múltiples**
   - Gradiente (fuerzas)
   - Cambio en energía
   - Cambio en geometría
   - Usar combinación de criterios

5. **Validación es esencial**
   - Comparar con datos experimentales
   - Verificar simetría molecular
   - Analizar múltiples conformaciones
   - Revisar conectividad química

6. **Para moléculas flexibles**
   - Generar múltiples conformaciones iniciales
   - Optimizar cada una
   - Identificar mínimos globales y locales
   - Considerar métodos de optimización global

### Recomendaciones Prácticas

| Tipo de Sistema | Método Recomendado | Consideraciones |
|-----------------|-------------------|-----------------|
| Moléculas rígidas (<50 átomos) | BFGS o L-BFGS | Convergencia rápida |
| Moléculas flexibles | Multi-inicio + L-BFGS | Múltiples conformaciones |
| Sistemas grandes (>1000 átomos) | L-BFGS con límite de memoria | Eficiencia memoria |
| Búsqueda global | Basin Hopping o DE | Exploración exhaustiva |
| Proteínas | L-BFGS con restricciones | Mantener estructura secundaria |
| Pre-optimización | Steepest Descent + UFF | Rápido y robusto |

### Flujo de Trabajo Recomendado

```
1. Preparación
   ↓
2. Generar geometría 3D inicial
   ↓
3. Pre-optimización (UFF, SD, 100-200 iter)
   ↓
4. Optimización fina (MMFF94, L-BFGS, criterios estrictos)
   ↓
5. Verificación (convergencia, simetría, energía razonable)
   ↓
6. (Si flexible) Generar múltiples conformaciones
   ↓
7. Análisis y validación
```

## 🔗 Referencias y Recursos Adicionales

### Artículos Fundamentales

1. **Schlegel, H. B.** (2011). *Geometry Optimization*. 
   - WIREs Comput. Mol. Sci., 1, 790-809. DOI: 10.1002/wcms.34

2. **Nocedal, J. & Wright, S. J.** (2006). *Numerical Optimization*, 2nd ed. 
   - Springer. ISBN: 978-0387303031

3. **Liu, D. C. & Nocedal, J.** (1989). *On the Limited Memory BFGS Method*. 
   - Math. Program., 45, 503-528. DOI: 10.1007/BF01589116

4. **Fletcher, R. & Reeves, C. M.** (1964). *Function Minimization by Conjugate Gradients*. 
   - Comput. J., 7, 149-154. DOI: 10.1093/comjnl/7.2.149

5. **Wales, D. J. & Doye, J. P. K.** (1997). *Global Optimization by Basin-Hopping*. 
   - J. Phys. Chem. A, 101, 5111-5116. DOI: 10.1021/jp970984n

6. **Storn, R. & Price, K.** (1997). *Differential Evolution*. 
   - J. Global Optim., 11, 341-359. DOI: 10.1023/A:1008202821328

### Libros Recomendados

1. **Jensen, F.** (2017). *Introduction to Computational Chemistry*, 3rd ed.
   - Wiley. Cap. 10-11: Geometry Optimization. ISBN: 978-1118825990

2. **Leach, A. R.** (2001). *Molecular Modelling: Principles and Applications*, 2nd ed.
   - Pearson. Cap. 7: Energy Minimization. ISBN: 978-0582382107

3. **Cramer, C. J.** (2004). *Essentials of Computational Chemistry*, 2nd ed.
   - Wiley. Cap. 9: Optimization Methods. ISBN: 978-0470091821

4. **Press, W. H. et al.** (2007). *Numerical Recipes*, 3rd ed.
   - Cambridge. Cap. 10: Minimization or Maximization of Functions

### Recursos Online

- **SciPy Optimization**: https://docs.scipy.org/doc/scipy/reference/optimize.html
- **RDKit Optimization**: https://www.rdkit.org/docs/source/rdkit.Chem.rdForceFieldHelpers.html
- **ASE Optimizers**: https://wiki.fysik.dtu.dk/ase/ase/optimize.html
- **Gaussian Optimization**: https://gaussian.com/opt/

### Software Relacionado

- **RDKit**: Optimización MM con MMFF94/UFF
- **Open Babel**: Múltiples campos de fuerza
- **ASE**: Interfaz unificada de optimización
- **Psi4**: Optimización QM
- **ORCA**: Optimización QM avanzada
- **Gaussian**: Suite completa de optimización

## 📝 Cuestionario de Autoevaluación

1. **¿Cuál es la diferencia entre un mínimo local y un mínimo global?**

2. **¿Por qué el método de Steepest Descent converge lentamente cerca del mínimo?**

3. **¿Qué ventaja tiene L-BFGS sobre BFGS para sistemas grandes?**

4. **¿Cuándo es necesario usar métodos de optimización global?**

5. **¿Qué información proporciona el Hessiano sobre un punto estacionario?**

6. **¿Por qué es importante usar múltiples geometrías iniciales para moléculas flexibles?**

7. **¿Cuál es la diferencia entre Simulated Annealing y Basin Hopping?**

8. **¿Qué criterios de convergencia son los más robustos?**

9. **¿Cómo se puede verificar que una optimización convergió correctamente?**

10. **¿Qué hacer si una optimización no converge después de muchas iteraciones?**

### Respuestas (Revisa después de intentar)

<details>
<summary>Haz clic para ver las respuestas</summary>

1. Mínimo local es el menor valor de energía en una región; mínimo global es el menor en toda la PES

2. Porque la dirección del gradiente es perpendicular a los contornos, causando zig-zag en valles estrechos

3. L-BFGS requiere memoria O(mN) en lugar de O(N²), almacenando solo los últimos m vectores

4. Cuando se sospecha que hay múltiples mínimos locales y se necesita el mínimo absoluto

5. Los autovalores indican si es mínimo (todos positivos), TS (1 negativo) o punto de silla (2+ negativos)

6. Para explorar diferentes cuencas de la PES y evitar quedar atrapado en mínimos locales altos

7. SA usa temperatura decreciente con MC; Basin Hopping hace optimización local después de cada paso MC

8. Combinación de: RMS del gradiente, máximo gradiente, cambio en energía y cambio en geometría

9. Verificar gradiente cercano a cero, energía estable, geometría química razonable, comparar con experimental

10. Cambiar método de optimización, mejorar geometría inicial, relajar criterios, o usar pre-optimización

</details>

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 3.3: Optimización de Geometrías Moleculares**

Ahora comprendes:
- ✅ Fundamentos matemáticos de la optimización
- ✅ Métodos locales: SD, CG, Newton, L-BFGS
- ✅ Métodos globales: MC, SA, Basin Hopping, GA, DE
- ✅ Criterios de convergencia
- ✅ Aplicación práctica a moléculas reales
- ✅ Problemas comunes y soluciones

**Siguiente actividad**: Superficies de Energía Potencial

[![Previous](https://img.shields.io/badge/⬅️_Actividad_3.2-Campos_de_Fuerza-blue.svg)](02_campos_fuerza_practica.ipynb)
[![Next](https://img.shields.io/badge/Actividad_3.4_➡️-Superficies_PES-green.svg)](04_superficies_energia.ipynb)

---

📚 **[Volver al Módulo 3](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G - 2026*

</div>